# Customer Question Classification for ISP

klasifikasi pertanyaan pelanggan ISP ke dalam tiga kategori:
- Information
- Request
- Problem

## Library

In [770]:
import pandas as pd
import re
import numpy as np
import nltk

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    classification_report
)


nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## upload csv

## Dataset Load
Dataset berisi pertanyaan pelanggan ISP dalam bentuk teks bebas.  

In [771]:
df = pd.read_csv("question_list.csv")
df = df.dropna(subset=["question_list"])
df = df.reset_index(drop=True)

df.head()

,question_list
0,"""Internet mati nih"""
1,"""Harga STB berapa ya?"""
2,"""Internet saya putus sambung"""
3,"""Mengapa saya tidak dapat mengisi formulir rel..."
4,"""mengapa ketika ingin melakukan pembayaran tid..."


In [772]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question_list  1999 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


## Stopwords removal
## Stopword Handling & Text Normalization

Stopword Bahasa Indonesia digunakan untuk menghilangkan kata-kata umum yang tidak memiliki makna semantik.
Selain itu, ditambahkan stopword khusus domain ISP dan normalisasi kata informal untuk mengurangi variasi bahasa.

In [773]:
with open("tala-stopwords-indonesia.txt", "r", encoding="utf-8") as f:
    tala_stopwords = set(line.strip() for line in f if line.strip())

isp_stopwords = {
    "pak", "bu", "kak", "min", "admin", "cs",
    "tolong", "mohon", "ya", "dong", "nih",
    "saya", "kami", "aku", "anda",
    "nya", "dong", "please"
}

stopword_list = tala_stopwords.union(isp_stopwords)
len(stopword_list)

767

### normalization
normalization map digunakan untuk menyatukan berbagai variasi kata informal/slang/teknis kedalam satu bentuk standar (canonical form) atau dalam NLP, ini disebut lexical normalization

In [774]:
NORMALIZATION_MAP = {
    "lemot": "lambat",
    "lelet": "lambat",
    "ngelag": "lambat",
    "wifi": "internet",
    "disconnect": "putus",
    "dc": "putus",
    "offline": "putus",
    "no internet": "tidak konek",
    "ga bisa": "tidak bisa",
    "gak bisa": "tidak bisa",
    "nggak": "tidak",
    "loss": "los"
}

### dampak ke model?
| Aspek           | Dampak       |
| --------------- | ------------ |
| Vocabulary size | Lebih kecil  |
| TF-IDF weight   | Lebih stabil |
| Recall Problem  | Naik         |
| Overfitting     | Turun        |


question, kenapa tidak di lakukan stemming?
dalam case ini spesifik, stemming bisa menghilangkan makna teknis seperti putus, diputus, memutus → ambigu. satu kata yang 'sama' bisa masuk ke class yang berbeda.

jadinya, Normalisasi lebih aman & terkontrol

In [775]:
def normalize_text(text):
    for k, v in NORMALIZATION_MAP.items():
        text = text.replace(k, v)
    return text


def preprocess_text(text):
    text = str(text).lower()
    text = normalize_text(text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopword_list]
    return " ".join(tokens)

## apply preprocessing

Tahap preprocessing mencakup:
1. Lowercasing
2. Normalisasi istilah informal
3. Penghapusan karakter non-alfabet
4. Tokenisasi
5. Stopword removal

Tujuan utama tahap ini adalah mengurangi noise tanpa menghilangkan informasi penting yang relevan untuk klasifikasi.

In [776]:
df["clean_question"] = df["question_list"].apply(preprocess_text)
df[["question_list", "clean_question"]].head(50)

,question_list,clean_question
0,"""Internet mati nih""",internet mati
1,"""Harga STB berapa ya?""",harga stb
2,"""Internet saya putus sambung""",internet putus sambung
3,"""Mengapa saya tidak dapat mengisi formulir rel...",mengisi formulir relokasi disubmit blank
4,"""mengapa ketika ingin melakukan pembayaran tid...",pembayaran diproses
5,"""bisa dibantu informasikan prosedur pergantian...",dibantu informasikan prosedur pergantian paket...
6,"""apakah untuk pergantian paket layanan bisa se...",pergantian paket layanan bundling promo tahunan
7,"""apakah jika saat proses pergantian paket laya...",proses pergantian paket layanan apps mybiznet ...
8,"""Internet mati""",internet mati
9,"""apakah order promo bisa diproses berkali-kali?""",order promo diproses berkali kali


### identify duplicate

In [777]:
dup_mask = df.duplicated(subset="clean_question", keep=False)
dup_df = df[dup_mask].sort_values("clean_question")

print("total duplicate rows:", len(dup_df))
dup_df[["question_list", "clean_question"]].head(20)

total duplicate rows: 656


,question_list,clean_question
1731,"""Saya mau bertanya""",
1618,"""Kok lama""",
1532,"""1000032151""",
1082,"""apakah bisa?""",
1703,"""Masih sama ini""",
942,"""Boleh tanya min?""",
1706,"""tadi katanya nggak ada""",
1406,"""Mau ke cs""",
1866,"""saya mau *12+5 *""",
1379,"""001000378036""",


### duplicate removal

In [778]:
before = len(df)

df = df.drop_duplicates(subset="clean_question")
df = df.reset_index(drop=True)

after = len(df)

print(f"before: {before}")
print(f"after: {after}")
print(f"removed: {before - after}")

before: 1999
after: 1532
removed: 467


### english words removal
ketika melihat dataset, ada beberapa kalimat dalam bahasa inggris yang dinilai tidak begitu relevan jika tetap disertakan dalam dataset, oleh karena itu beberapa kalimat bahasa inggris yang kurang relevan dihapus berdasarkan beberapa kata spesifik, seperti yang ada pada 'common_english_words'

In [779]:
def is_english(text, threshold=0.6):
    words = text.split()
    if len(words) < 3:
        return False

    ascii_ratio = sum(c.isascii() for c in text) / len(text)

    common_english_words = {
        "the", "and", "to", "is", "are", "you", "please", "want", "my", "can"
    }

    eng_word_count = sum(1 for w in words if w in common_english_words)

    return (ascii_ratio > threshold) and (eng_word_count >= 2)

### english words removal

In [780]:
english_df = df[df["clean_question"].apply(is_english)]

print("detected:", len(english_df))
english_df[["question_list", "clean_question"]].head(20)

detected: 11


,question_list,clean_question
192,"""can you speak english?""",can you speak english
194,"""Hai, my connection internet is so bad, can yo...",hai my connection internet is so bad can you h...
736,"""i want to buy extra quota""",i want to buy extra quota
742,"""i want to buy quota""",i want to buy quota
747,"""i want to buy promo""",i want to buy promo
750,"""i want to relocation""",i want to relocation
1175,"""i wanna check coverage area, can you help me?""",i wan na check coverage area can you help me
1470,"""please check my account 5501147 , I forgot t...",check my account i forgot the service
1480,"""waaaa you only can check based on billing num...",waaaa you only can check based on billing number
1491,"""No, I just want to chat with you Bella""",no i just want to chat with you bella


In [781]:
before = len(df)

df = df[~df["clean_question"].apply(is_english)]
df = df.reset_index(drop=True)

after = len(df)

print(f"before eng remove: {before}")
print(f"after eng remove: {after}")
print(f"removed: {before - after}")

before eng remove: 1532
after eng remove: 1521
removed: 11


### check count dataset after duplicate removal + english removal

In [782]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521 entries, 0 to 1520
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_list   1521 non-null   object
 1   clean_question  1521 non-null   object
dtypes: object(2)
memory usage: 23.9+ KB


,question_list,clean_question
0,"""Internet mati nih""",internet mati
1,"""Harga STB berapa ya?""",harga stb
2,"""Internet saya putus sambung""",internet putus sambung
3,"""Mengapa saya tidak dapat mengisi formulir rel...",mengisi formulir relokasi disubmit blank
4,"""mengapa ketika ingin melakukan pembayaran tid...",pembayaran diproses


## Feature Engineering
TF-IDF digunakan untuk mengubah teks menjadi representasi numerik.
Pendekatan ini menimbang kata berdasarkan frekuensi lokal dan global, sehingga kata yang informatif memiliki bobot lebih besar.

Bigram digunakan untuk menangkap konteks frasa penting seperti permintaan layanan dan pelaporan gangguan.

### fungsi keyword dalam kategori: 
Sebagai sinyal semantik berbasis aturan (rule-based signals) untuk membantu proses auto-labeling dan peningkatan kualitas dataset.

Dataset awal:
Tidak berlabel,
Bahasa customer bebas & ambigu,
Label minoritas (Request, Problem) sulit ditangkap

➡️ Keyword list digunakan untuk:

Memberi indikasi awal kelas,
Mengurangi noise auto-labeling,
Membantu model mengenali pola semantik

In [783]:
PROBLEM_KEYWORDS = [
    "lambat", "putus", "mati", "tidak konek", "error",
    "gangguan", "rusak", "lampu merah", "stabil", 
    "hilang", "ilang", "aneh", "kendala", "lemot",
    "buffering", "loading", "down", "kendala", "ping", 
    "los", "masalah", "bermasalah", "down"
]

REQUEST_KEYWORDS = [
    "pasang", "pasang baru", "pindah", "relokasi",
    "ganti", "upgrade", "reset", "daftar",
    "teknisi", "tolong bantu", "dibantu",
    "minta", "ajukan", "buatkan", "cekkan"
]

INFO_KEYWORDS = [
    "cek", "tagihan", "bayar", "harga",
    "biaya", "promo", "paket", "informasi"
    "kenapa", "apakah", "berapa", "kapan", "dimana"
]


### fungsi category text.
Sebagai “semantic anchor text” untuk melakukan similarity-based auto labeling
Artinya: Ini contoh kalimat representatif (prototype), Dipakai untuk mengukur kemiripan makna, bukan sekadar kata yang sama.

!!! MASALAH YANG DISELESAIKAN !!!

Keyword-based labeling:
Rentan false positive,
Tidak paham konteks.

Contoh:
"kenapa internet saya lambat?"

Keyword:
kenapa → Information ❌,
lambat → Problem ✅

➡️ terjadi konflik

#### Category Anchor Text for Semantic Labeling

Setiap kategori didefinisikan menggunakan kumpulan kalimat representatif yang berfungsi sebagai anchor semantik.
Pendekatan ini memungkinkan pelabelan otomatis berbasis kemiripan makna, bukan sekadar pencocokan kata, sehingga lebih robust terhadap variasi bahasa pelanggan.


In [784]:
CATEGORY_TEXT = {
    "Problem": [
        "internet lambat",
        "internet sering putus",
        "internet mati",
        "tidak konek internet",
        "modem rusak",
        "lampu merah modem",
        "koneksi tidak stabil",
        "internet rumah bermasalah",
        "wifi sering hilang",
        "jaringan tidak normal",
        "koneksi lambat sekali",
        "internet down",
        "modem loss",
        "tidak masuk"
    ],
    "Request": [
        "pasang baru internet",
        "pindah alamat pemasangan",
        "ganti paket internet",
        "reset password internet",
        "kunjungan teknisi", 
        "tolong bantu pasang",
        "minta teknisi datang",
        "dibantu reset"
    ],
    "Information": [
        "cek tagihan internet",
        "informasi pembayaran",
        "cara bayar internet",
        "harga paket internet",
        "promo internet", 
        "kenapa tagihan naik",
        "berapa biaya pasang",
        "apakah ada promo"
    ]
}

## Vectorizer

In [785]:
corpus = df["clean_question"].tolist()
for texts in CATEGORY_TEXT.values():
    corpus.extend(texts)
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=1,
    sublinear_tf=True
)
vectorizer.fit(corpus)

TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)

In [786]:
category_vectors = {}

for cat, texts in CATEGORY_TEXT.items():
    vecs = vectorizer.transform(texts).toarray()
    category_vectors[cat] = vecs.mean(axis=0)

## Auto Labelling
### Auto Labeling Strategy — Technical Explanation

Auto labeling adalah proses pemberian label secara semi-otomatis pada data teks yang belum berlabel dengan memanfaatkan sinyal lemah (weak supervision) seperti aturan linguistik, keyword domain, dan kemiripan semantik. Pendekatan ini digunakan untuk membangun dataset berlabel awal (bootstrap dataset) secara efisien sebelum melatih model supervised learning.

#### Motivation
Pada data pertanyaan pelanggan ISP, anotasi manual penuh tidak praktis karena bahasa bersifat informal, bervariasi, dan dataset tidak seimbang. Auto labeling memungkinkan percepatan proses pelabelan dengan biaya rendah, sekaligus menjaga konsistensi label.

#### Preprocessing & Normalization
Normalisasi teks dilakukan untuk menyatukan berbagai variasi slang, istilah teknis, dan typo ke dalam bentuk standar. Langkah ini penting karena auto labeling sangat sensitif terhadap variasi leksikal; tanpa normalisasi, sinyal semantik akan terpecah dan menurunkan kualitas label.

#### Keyword-based Weak Signals
Daftar keyword per kategori (Problem, Request, Information) digunakan sebagai sinyal awal untuk mendeteksi indikasi kelas. Pendekatan ini memiliki recall tinggi namun rentan terhadap ambiguitas konteks, sehingga tidak digunakan sebagai keputusan akhir.

#### Semantic Anchor Text (CATEGORY_TEXT)
Setiap kategori didefinisikan menggunakan kumpulan kalimat representatif (anchor text) yang menggambarkan makna inti kategori tersebut. Kalimat ini berfungsi sebagai prototipe semantik untuk membandingkan pertanyaan pelanggan berdasarkan kemiripan makna, bukan sekadar kecocokan kata.

#### Similarity-based Label Assignment
Teks pelanggan dan anchor text direpresentasikan dalam ruang vektor menggunakan TF-IDF. Label diberikan berdasarkan nilai cosine similarity tertinggi terhadap anchor kategori. Pendekatan ini memungkinkan pemahaman konteks kalimat dan mengurangi false positive yang umum terjadi pada metode berbasis keyword.

#### Confidence Thresholding
Threshold kemiripan diterapkan untuk mengontrol kualitas label. Pertanyaan dengan skor di bawah threshold dibiarkan tidak berlabel (NaN) untuk menghindari noise berlebihan. Strategi ini memastikan hanya data dengan tingkat keyakinan tinggi yang digunakan pada tahap supervised learning.

#### Hybrid Weak Supervision
Auto labeling menggabungkan kelebihan keyword-based labeling (cakupan luas) dan similarity-based labeling (presisi tinggi). Kombinasi ini meningkatkan keseimbangan antara coverage dan quality, khususnya pada kelas minoritas seperti Request dan Problem.

#### ML Pipeline
Auto labeling bukan tujuan akhir, melainkan tahap awal untuk menghasilkan dataset berlabel yang cukup baik. Kualitas auto-labeling dievaluasi secara tidak langsung melalui performa model supervised downstream (Linear SVM). Stabilitas metrik presisi dan recall macro menunjukkan efektivitas strategi auto labeling.

### Conclusion
Pendekatan auto labeling ini dipilih karena sesuai dengan karakteristik data layanan pelanggan yang tidak terstruktur dan tidak seimbang. Dengan mengombinasikan normalisasi, keyword, anchor semantik, dan thresholding, strategi ini mampu menghasilkan dataset berlabel yang layak untuk pelatihan model klasifikasi teks secara robust dan terukur.


In [787]:
LABEL_MAP = {
    "Information": 0,
    "Request": 1,
    "Problem": 2
}

In [788]:
def auto_label_hybrid(text, threshold=0.04):
    tokens = text.split()

    if len(tokens) <= 3:
        return 0  

    for kw in PROBLEM_KEYWORDS:
        if kw in text:
            return 2

    for kw in REQUEST_KEYWORDS:
        if kw in text:
            return 1

    for kw in INFO_KEYWORDS:
        if kw in text:
            return 0

    text_vec = vectorizer.transform([text]).toarray()[0]

    scores = {}
    for cat, cat_vec in category_vectors.items():
        score = cosine_similarity(
            text_vec.reshape(1, -1),
            cat_vec.reshape(1, -1)
        )[0][0]
        scores[cat] = score

    best_cat = max(scores, key=scores.get)

    if scores[best_cat] >= threshold:
        return LABEL_MAP[best_cat]

    return 0

In [789]:
df["label"] = df["clean_question"].apply(auto_label_hybrid)

### check label value

In [790]:
df["label"].value_counts(dropna=False)

label
0    1250
2     141
1     130
Name: count, dtype: int64

### check sample

In [791]:
sample_df = pd.concat([
    df[df["label"] == 0].sample(850, random_state=42),
    df[df["label"] == 1].sample(120, random_state=42),
    df[df["label"] == 2].sample(120, random_state=42),
])

sample_df = sample_df[["question_list", "clean_question", "label"]]
sample_df


,question_list,clean_question,label
842,"""ok pakte apa yang kamu sarankan?""",ok pakte sarankan,0
1344,"""bagaimana cara untuk pemutusan jaringan?""",pemutusan jaringan,0
466,"""bagaimana cari downgrade?""",cari downgrade,0
1146,"""saya mau cek jumlah perangkat yg terhubung ke...",cek perangkat yg terhubung modem,0
596,"""mau ubah status modem jadi buy min""",ubah status modem buy,0
...,...,...,...
1480,"""internet saya slow pas beut nich. ads yang bi...",internet slow pas beut nich ads tangani gk kawan,2
656,"""Min tolong kirimkan teknisi kelokasi, modem s...",kirimkan teknisi kelokasi modem los,2
679,"""layanan saya Biznet 3D, saat akses game cukup...",layanan biznet d akses game lambat akses youtu...,2
1275,"""Siang, saya Ganjar saya mau berlangganan laya...",siang ganjar berlangganan layanan internet tv ...,2


In [ ]:
sample_df.to_csv("check_sample", index=False) 

## Classification - Linear SVM

Pemilihan **Linear Support Vector Machine (Linear SVM)** pada tahap klasifikasi dilakukan berdasarkan karakteristik data, representasi fitur, serta tujuan sistem, bukan semata-mata karena performa empiris. Berikut penjelasan teknis dan konseptualnya.

### Nature of the Feature Space
Teks pertanyaan pelanggan direpresentasikan menggunakan **TF-IDF**, yang menghasilkan:
- Dimensi sangat tinggi (ribuan fitur)
- Matriks sparse
- Hubungan fitur yang hampir linear secara semantik

Dalam ruang vektor TF-IDF, kelas teks cenderung **linearly separable atau hampir separable**, sehingga model linear menjadi pilihan yang secara teoritis tepat.

### Maximum Margin Principle
Linear SVM bekerja dengan mencari **hyperplane yang memaksimalkan margin** antar kelas. Prinsip ini memberikan:
- Generalisasi yang lebih baik
- Ketahanan terhadap noise label (yang umum pada auto-labeling)
- Stabilitas performa pada dataset kecil hingga menengah

Margin maksimum sangat penting ketika label tidak sepenuhnya bersih.

### Robustness to Noisy and Imbalanced Data
Auto-labeled data mengandung **label noise terkontrol**. Linear SVM:
- Tidak mencoba memodelkan probabilitas
- Tidak memaksakan fit pada setiap titik data
- Fokus pada support vectors (data paling informatif)

Hal ini membuatnya lebih tahan terhadap kesalahan label dibanding model probabilistik seperti Naive Bayes atau Logistic Regression.

### Bias-Variance Trade-off
Linear SVM memiliki:
- Bias relatif tinggi
- Variance rendah

Kombinasi ini ideal untuk:
- Dataset berukuran ~1–5k
- Feature space besar
- Risiko overfitting yang harus ditekan

Model non-linear (RBF SVM, neural networks) memiliki variance lebih tinggi dan berpotensi overfit pada data noisy.

### Computational Efficiency
Dengan solver linear (`LinearSVC`):
- Training time hampir linear terhadap jumlah sampel
- Memory efficient untuk sparse matrix
- Scalable untuk iterasi dan cross-validation

Ini penting untuk eksperimen cepat dan tuning pipeline auto-labeling.

### Interpretability
Bobot koefisien Linear SVM dapat dianalisis untuk:
- Melihat fitur (kata/bigram) paling berpengaruh
- Melakukan error analysis
- Memvalidasi konsistensi domain knowledge

Interpretabilitas ini krusial dalam konteks industri dan evaluasi kualitas dataset.

### Compatibility with Cross-Validation
Linear SVM stabil terhadap variasi data fold, sehingga:
- Cocok untuk Stratified K-Fold
- Memberikan estimasi performa yang konsisten
- Memudahkan deteksi overfitting akibat data leakage atau bias label

### Why Not Other Models?
- **Naive Bayes**: asumsi independensi fitur terlalu kuat, sensitif terhadap keyword noise
- **Logistic Regression**: mirip Linear SVM, tetapi kurang robust terhadap outlier
- **Tree-based models**: tidak cocok untuk sparse high-dimensional text
- **Deep Learning**: membutuhkan data besar dan label bersih

### Role in the Overall System
Linear SVM berfungsi sebagai:
- Evaluator kualitas auto-labeling
- Baseline classifier yang kuat
- Model produksi yang sederhana dan stabil

### Conclusion
Linear SVM dipilih karena secara matematis dan praktis paling sesuai untuk klasifikasi teks berbasis TF-IDF dengan dataset berlabel otomatis. Model ini memberikan keseimbangan optimal antara generalisasi, robustness terhadap noise, efisiensi komputasi, dan interpretabilitas, sehingga menjadi pilihan rasional dalam pipeline ini.


In [793]:
X = df["clean_question"]
y = df["label"]


## TF-IDF Vectorizer Configuration — Technical Explanation

TF-IDF (Term Frequency–Inverse Document Frequency) digunakan untuk mengubah teks pertanyaan pelanggan menjadi representasi numerik yang dapat diproses oleh model machine learning. Konfigurasi berikut dipilih untuk menyeimbangkan kekayaan fitur, noise reduction, dan stabilitas model.

### ngram_range = (1, 2)
Parameter ini menentukan bahwa fitur yang diekstraksi mencakup:
- Unigram (satu kata)
- Bigram (dua kata berurutan)

Penggunaan bigram sangat penting dalam domain ISP karena banyak makna kunci muncul sebagai frasa, bukan kata tunggal, seperti:
- "internet lambat"
- "lampu merah"
- "tidak konek"
- "pasang baru"

Bigram memungkinkan model menangkap konteks lokal dan mengurangi ambiguitas yang sering terjadi pada unigram.

### min_df = 2
Parameter ini menghapus kata atau n-gram yang hanya muncul di kurang dari 2 dokumen.

Tujuan:
- Menghilangkan typo unik
- Mengurangi noise
- Menurunkan dimensi fitur yang tidak informatif

Pada dataset customer service, banyak kata muncul sekali akibat kesalahan ketik atau nama spesifik yang tidak relevan untuk klasifikasi.

### max_df = 0.9
Parameter ini menghapus fitur yang muncul di lebih dari 90% dokumen.

Tujuan:
- Menghilangkan kata yang terlalu umum
- Menghindari fitur yang tidak diskriminatif

Kata seperti "internet", "saya", atau "tolong" muncul hampir di semua pertanyaan dan tidak membantu membedakan kelas.

### sublinear_tf = True
Dengan pengaturan ini, frekuensi term dihitung menggunakan skala logaritmik:


In [794]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

## split 80-20 

In [795]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train size:", len(X_train))
print("Test size :", len(X_test))

Train size: 1216
Test size : 305


In [796]:
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

## Linear SVM Model Configuration

Model klasifikasi yang digunakan adalah **Linear Support Vector Machine (LinearSVC)**, yang dirancang untuk bekerja optimal pada data berdimensi tinggi dan sparse seperti representasi TF-IDF.

### class_weight = "balanced"
Parameter ini digunakan untuk menangani **ketidakseimbangan kelas (class imbalance)** pada dataset.

Secara default, SVM menganggap semua kelas memiliki bobot yang sama. Pada dataset layanan pelanggan ISP, distribusi label tidak seimbang (kelas Information jauh lebih dominan dibanding Request dan Problem). Dengan pengaturan ini, bobot setiap kelas dihitung secara otomatis berdasarkan formula:

weight_i = n_samples / (n_classes * n_samples_i)


Efeknya:
- Kesalahan pada kelas minoritas diberi penalti lebih besar
- Model dipaksa untuk lebih memperhatikan kelas Request dan Problem
- Recall kelas minoritas meningkat tanpa harus melakukan oversampling

### C = 1.0 (Regularization Strength)
Parameter **C** mengontrol trade-off antara:
- Margin yang lebar (generalization)
- Kesalahan klasifikasi pada data training

Interpretasi:
- C kecil → margin lebih lebar, toleransi error tinggi
- C besar → margin sempit, risiko overfitting meningkat

Nilai C=1.0 dipilih sebagai titik keseimbangan default yang stabil, terutama ketika data mengandung label noise akibat auto-labeling. Nilai ini mencegah model terlalu agresif mempelajari noise.

### random_state = 42
Parameter ini memastikan **reproducibility** dari hasil training, terutama ketika:
- Data di-shuffle
- Proses training diulang pada environment berbeda

Dengan random_state tetap, eksperimen dapat direplikasi secara konsisten.

### Model Training
```python
svm_model.fit(X_train_vec, y_train)

In [797]:
svm_model = LinearSVC(
    class_weight="balanced",
    C=1.0,
    random_state=42
)

svm_model.fit(X_train_vec, y_train)


c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(class_weight='balanced', random_state=42)

In [798]:
y_pred = svm_model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average="macro")

print(f"accuracy : {accuracy:.4f}")
print(f"precision: {precision:.4f}")
print(f"recall   : {recall:.4f}")

accuracy : 0.7869
precision: 0.5770
recall   : 0.6479


## Stratified K-Fold Cross Validation—

Stratified K-Fold Cross Validation digunakan untuk mengevaluasi performa model secara lebih stabil dan representatif, khususnya pada dataset dengan distribusi kelas yang tidak seimbang.

### StratifiedKFold
Berbeda dengan K-Fold biasa, StratifiedKFold memastikan bahwa **proporsi setiap kelas pada setiap fold mendekati distribusi kelas pada dataset keseluruhan**. Hal ini sangat penting pada kasus klasifikasi pertanyaan pelanggan ISP, di mana kelas Information jauh lebih dominan dibanding Request dan Problem.

Tanpa stratifikasi, beberapa fold berpotensi kehilangan data dari kelas minoritas, yang akan menghasilkan estimasi performa yang bias dan tidak stabil.

### n_splits = 15
Parameter ini menentukan jumlah fold (partisi) data.

Interpretasi teknis:
- Dataset dibagi menjadi 15 bagian
- Setiap iterasi menggunakan 14 bagian sebagai data latih dan 1 bagian sebagai data uji
- Proses ini diulang 15 kali sehingga setiap data pernah menjadi data uji tepat satu kali

Alasan memilih nilai 15:
- Dataset berukuran relatif kecil (~1–2k sampel)
- Jumlah fold lebih besar memberikan estimasi performa yang lebih halus
- Mengurangi variansi evaluasi akibat satu pembagian data tertentu

Namun, jumlah fold yang terlalu besar dapat meningkatkan waktu komputasi dan risiko overfitting evaluasi. Nilai 15 dipilih sebagai kompromi antara stabilitas dan efisiensi.

### shuffle = True
Data diacak sebelum dibagi ke dalam fold untuk:
- Menghindari bias urutan data
- Mencegah data serupa terkumpul dalam satu fold
- Memastikan distribusi sampel yang lebih representatif

### random_state = 42
Parameter ini memastikan bahwa proses pengacakan dapat direproduksi. Dengan random_state tetap, hasil evaluasi akan konsisten ketika eksperimen dijalankan ulang.

### Why Stratified K-Fold Instead of Single Train-Test Split?
- Mengurangi ketergantungan pada satu pembagian data (misalnya 80–20)
- Memberikan estimasi performa yang lebih reliabel
- Sangat cocok untuk dataset auto-labeled yang mengandung noise

### Role in the Pipeline
Stratified K-Fold digunakan sebagai:
- Mekanisme evaluasi model
- Validasi kualitas auto-labeling
- Indikator kestabilan model terhadap variasi data

### Conclusion
Penggunaan StratifiedKFold dengan n_splits=15 memberikan evaluasi performa yang adil, stabil, dan representatif untuk klasifikasi teks pada dataset yang tidak seimbang. Pendekatan ini membantu memastikan bahwa metrik yang diperoleh mencerminkan kemampuan generalisasi model dengan baik.


In [799]:
skf = StratifiedKFold(
    n_splits=15,
    shuffle=True,
    random_state=42
)

In [ ]:
accuracy_scores = []
precision_scores = []
recall_scores = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    model = LinearSVC(
        class_weight="balanced",
        C=1.0,
        random_state=42
    )

    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="macro")
    rec = recall_score(y_test, y_pred, average="macro")

    accuracy_scores.append(acc)
    precision_scores.append(prec)
    recall_scores.append(rec)

    print(f"accuracy: {acc:.4f}")
    print(f"precision: {prec:.4f}")
    print(f"recall: {rec:.4f}")


accuracy : 0.8529
precision: 0.6883
recall   : 0.7393
accuracy : 0.7843
precision: 0.5789
recall   : 0.6528
accuracy : 0.8627
precision: 0.6944
recall   : 0.7139
accuracy : 0.8235
precision: 0.6286
recall   : 0.6687
accuracy : 0.8235
precision: 0.6160
recall   : 0.6310


c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users

accuracy : 0.8235
precision: 0.6552
recall   : 0.6234
accuracy : 0.8119
precision: 0.6153
recall   : 0.6265
accuracy : 0.9010
precision: 0.7740
recall   : 0.7287


c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


accuracy : 0.7327
precision: 0.5251
recall   : 0.6274
accuracy : 0.8812
precision: 0.7789
recall   : 0.6876
accuracy : 0.8317
precision: 0.6517
recall   : 0.6015
accuracy : 0.7921
precision: 0.5628
recall   : 0.5524
accuracy : 0.8416
precision: 0.6581
recall   : 0.7376


c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users

accuracy : 0.8416
precision: 0.6648
recall   : 0.7046
accuracy : 0.8515
precision: 0.7009
recall   : 0.7747


c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\ilham\miniconda3\envs\modsim\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


## Final result

In [ ]:
print(f"accuracy: {np.mean(accuracy_scores):.4f}")
print(f"precision: {np.mean(precision_scores):.4f}")
print(f"recall: {np.mean(recall_scores):.4f}")


Accuracy  : 0.8304
Precision : 0.6529
Recall    : 0.6713


## Model Evaluation

### Final Metrics
- **Accuracy  : 0.8304**
- **Precision : 0.6529**
- **Recall    : 0.6713**

Evaluasi dilakukan menggunakan Stratified K-Fold Cross Validation untuk memastikan metrik yang stabil dan representatif terhadap distribusi kelas yang tidak seimbang.

---

### Accuracy (83.04%)
Accuracy menunjukkan proporsi total prediksi yang benar terhadap seluruh data.

Nilai ini relatif tinggi karena:
- Kelas mayoritas (Information) terklasifikasi dengan baik
- Model linear mampu menangkap pola dominan pada TF-IDF space

Namun, pada dataset yang tidak seimbang, accuracy **bukan metrik utama**, karena model bisa tampak “bagus” dengan hanya memprediksi kelas mayoritas.

---

### Precision (65.29%)
Precision mengukur seberapa banyak prediksi positif yang benar.

Interpretasi:
- Ketika model memprediksi suatu pertanyaan sebagai *Request* atau *Problem*, sekitar **65% prediksi tersebut benar**
- Masih terdapat false positive, terutama antara kelas Request dan Information

Precision yang tidak terlalu tinggi ini mencerminkan:
- Overlap semantik antar kelas
- Ambiguitas bahasa pelanggan
- Noise dari auto-labeling

Namun, nilai ini masih **sangat wajar** untuk data auto-labeled tanpa anotasi manual penuh.

---

### Recall (67.13%)
Recall mengukur seberapa banyak data aktual dari suatu kelas yang berhasil terdeteksi oleh model.

Interpretasi:
- Model berhasil menangkap sekitar **67% pertanyaan Request dan Problem yang sebenarnya**
- Recall yang lebih tinggi dari precision menunjukkan bahwa model **lebih sensitif daripada konservatif**

Dalam konteks layanan pelanggan ISP, recall yang relatif lebih tinggi sering lebih diutamakan karena:
- Lebih baik mendeteksi potensi masalah pelanggan
- Risiko miss (false negative) lebih kritis dibanding false alarm

---

### Precision–Recall Trade-off
Perbedaan antara precision dan recall menunjukkan bahwa model:
- Cenderung menangkap lebih banyak kasus minoritas
- Mengorbankan sedikit ketepatan demi cakupan

Ini merupakan **trade-off yang disengaja**, terutama dengan penggunaan:
- `class_weight="balanced"`
- Auto-labeled dataset

---

### Impact of Auto-Labeling
Metrik ini harus dievaluasi dengan mempertimbangkan bahwa:
- Label training mengandung noise
- Ground truth bukan hasil anotasi manusia sepenuhnya

Dalam konteks ini:
> Precision ~65% dan Recall ~67% menunjukkan kualitas auto-labeling yang cukup baik untuk training model supervised.

Model tidak hanya menghafal noise, tetapi mampu mempelajari pola umum.

---

### Model Stability
Konsistensi metrik pada cross-validation menunjukkan bahwa:
- Model tidak overfit pada satu subset data
- Representasi TF-IDF + Linear SVM stabil
- Pipeline preprocessing efektif

---

### Engineering Assessment
Dari sudut pandang AI Engineer:
- Model layak sebagai baseline produksi
- Auto-labeling strategy terbukti efektif
- Pipeline scalable dan interpretable


---

### How to Improve Further
Beberapa langkah lanjutan:
- Manual review pada subset minoritas
- Active learning loop
- Threshold tuning untuk precision/recall control
- Penambahan anchor text & normalization

---

### Conclusion
Hasil evaluasi menunjukkan bahwa pendekatan auto-labeling yang digunakan mampu menghasilkan dataset berlabel yang cukup berkualitas. Linear SVM yang dilatih pada data tersebut memberikan performa yang stabil dan seimbang, menjadikannya solusi yang rasional dan dapat dipertanggungjawabkan secara teknis dalam konteks klasifikasi pertanyaan pelanggan ISP.
